In [ ]:
import math 
import matplotlib.patches as mpatches

In [ ]:
import jsonlines
import os
import datetime
import json as js
import sqlite3 as sql
import statistics

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sqlite3

# next command ensures that plots appear inside the notebook
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  # also improves the look of plots
sns.set()
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from palpatine import Sentiment

In [ ]:
def export_to_sql(input_stuff, output):
    """ 
    Export data to an SQLite database. Creates 4 tables (main, users, replies, hashtags) and uses the attributes \n
    from the JSON file.

    Attributes
    directory - Absolute directory of the folder with the files to be added
    output - Name of the output file (.db)
    """

    con = sql.connect(f'{output}.db')
    vader = SentimentIntensityAnalyzer()
    palpatine = Sentiment()

    # Lists for each table

    # main
    tweet_id, user_id, tweet_time, tweet_text, favourites, retweets, truncated, tweet_lang, sent = [[] for i in range(9)]

    # users
    screen_name, user_created, user_lang, user_desc, verified, followers, default_prof = [[] for i in range(7)]
    default_prof_image, username = [[] for i in range(2)]

    # replies
    reply_id, reply_user = [[] for i in range(2)]

    # hashtags
    hashtags = []

    def get_attrib(file):
        with jsonlines.open(file, mode='r') as main:
            for line in main.iter(allow_none=True, skip_invalid=True, skip_empty=True):
                
                # Remove deleted messages from stream
                if "created_at" not in line:
                    continue
                
                # Table 1 (main)
                tweet_id.append(line['id_str'])
                user_id.append(line['user']['id_str'])
                tweet_time.append(datetime.datetime.strptime(line['created_at'], '%a %b %d %H:%M:%S %z %Y'))
                favourites.append(line['favorite_count'])
                retweets.append(line['retweet_count'])
                truncated.append(line['truncated'])
                text = line['text']
                tweet_text.append(text)
                
                if 'lang' in line:
                    lang = line['lang']
                    tweet_lang.append(lang)
                    if lang == 'en':
                        score = vader.polarity_scores(text)['compound']
                        sent.append(score)
                    else:
                        score = palpatine.basic_polarity(tweet=text, language=lang)
                        if isinstance(score, dict):
                            sent.append(score['compound'])
                        else:
                            sent.append(score)

                else:
                    tweet_lang.append('N/A')
                    sent.append('N/A') 

                # Table 2 (users)
                screen_name.append(line['user']['screen_name'])
                user_created.append(datetime.datetime.strptime(line['user']['created_at'], '%a %b %d %H:%M:%S %z %Y').date())
                user_lang.append(line['user']['lang'])
                user_desc.append(line['user']['description'])
                verified.append(line['user']['verified'])
                followers.append(line['user']['followers_count'])
                default_prof.append(line['user']['default_profile'])
                default_prof_image.append(line['user']['default_profile_image'])
                username.append(line['user']['name'])

                # Table 3 (replies)
                reply_id.append(line['in_reply_to_status_id_str'])
                reply_user.append(line['in_reply_to_user_id_str'])

                # Table 4 (hashtags)
                temp_3 = [x['text'] for x in line['entities']['hashtags']]
                if len(temp_3) == 0:
                    hashtags.append(None)
                else:
                    hashtags.append(temp_3)
        
    if os.path.isdir(input_stuff):
        files = [f'{input_stuff}/{x}' for x in os.listdir(input_stuff)]
        for file in files:
            get_attrib(file)
    else:
        get_attrib(input_stuff)

    main_dict = {
        'ID': tweet_id,
        'User ID': user_id,
        'Created At': tweet_time,
        'Text': tweet_text,
        'Language': tweet_lang,
        'Sentiment': sent,
        'Favourites': favourites,
        'Retweets': retweets,
        'Truncated': truncated
    }

    users_dict = {
        'User ID': user_id,
        'Date Created': user_created,
        'Screen Name': screen_name,
        'Name': username,
        'Description': user_desc,
        'Verified': verified,
        'Follower Count': followers,
        'Language': user_lang,
        'Default Profile': default_prof,
        'Default Profile Image': default_prof_image

    }

    temp_1, temp_2, temp_3 = [[] for i in range(3)]

    for x, v, z in zip(tweet_id, reply_id, reply_user):
        if v is None:
            continue
        temp_1.append(x)
        temp_2.append(v)
        temp_3.append(z)

    replies_dict = {
        'ID': temp_1,
        'Reply ID': temp_2,
        'Reply User': temp_3
    }

    temp_1 = [(i, x) for i, v in zip(tweet_id, hashtags) if v is not None for x in v]

    hashtags_dict = {
        'ID': [x[0] for x in temp_1],
        'Hashtag': [x[1] for x in temp_1]
    }

    # Dataframes
    df_main = pd.DataFrame(main_dict)
    df_users = pd.DataFrame(users_dict)
    df_replies = pd.DataFrame(replies_dict)
    df_hashtags = pd.DataFrame(hashtags_dict)

    # Dataframe cleaning
    df_main.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    df_hashtags.drop_duplicates(keep='last', inplace=True)
    df_replies.drop_duplicates(subset=['ID'], keep='last', inplace=True)
    df_users.drop_duplicates(subset=['User ID'], keep='last', inplace=True)

    # Schemas (Don't worry about it, you don't really need to know what this does)
    dtypes_1 = {
        'ID': 'TEXT',
        'User ID': 'TEXT',
        'Created At': 'TEXT',
        'Language': 'TEXT',
        'Text': 'TEXT',
        'Sentiment': 'REAL',
        'Retweets': 'INT',
        'Favourites': 'INT',
        'Truncated': 'INT'
    }

    dtypes_2 = {
        'User ID': 'TEXT',
        'Screen Name': 'TEXT',
        'Name': 'TEXT',
        'Date Created': 'NUMERIC',
        'Description': 'TEXT',
        'Verified': 'INT',
        'Follower Count': 'INT',
        'Language': 'TEXT',
        'Default Profile': 'INT',
        'Default Profile Image': 'INT'
    }

    dtypes_3 = {
        'ID': 'TEXT',
        'Reply ID': 'TEXT',
        'Reply User': 'TEXT'
    }

    dtypes_4 = {
        'ID': 'TEXT',
        'Hashtag': 'TEXT'
    }

    df_main.to_sql('main', con, if_exists='append', index=False, dtype=dtypes_1)
    df_users.to_sql('users', con, if_exists='append', index=False, dtype=dtypes_2)
    df_replies.to_sql('replies', con, if_exists='append', index=False, dtype=dtypes_3)
    df_hashtags.to_sql('hashtag', con, if_exists='append', index=False, dtype=dtypes_4)

    con.close()

directory = input('Enter the absolute directory of the folder with the files: ')
output_file = input('Enter the name of the database: ')

export_to_sql(directory, output_file)

print('Done')


In [ ]:
conn = sql.connect("demo.db")

In [ ]:
pd.read_sql_query("SELECT * FROM main", conn)

In [ ]:
all_tweets = """
SELECT *
FROM  main 
"""
df_complete = pd.read_sql_query(all_tweets, conn)
time_dict = df_complete.set_index('ID')['Created At'].to_dict()
tweet_user_dict = df_complete.set_index('ID')['User ID'].to_dict()

In [ ]:
query_BA_total = """
SELECT Date("Created At") as date, count(Text) as tweet_volume, main.*
FROM main
WHERE text LIKE \"%British_Airways%\"
GROUP BY Date("Created At")
"""

query_KLM_total = """
SELECT Date("Created At") as date, count(Text) as tweet_volume, main.*
FROM main
WHERE text LIKE \"%KLM%\" 
GROUP BY Date("Created At")
"""

df_BA_complete = pd.read_sql_query(query_BA_total, conn)
df_KLM_complete = pd.read_sql_query(query_KLM_total, conn)

In [ ]:
query_BA_total = """
SELECT *
FROM main
WHERE text LIKE \"%British_Airways%\"
"""

query_KLM_total = """
SELECT *
FROM main
WHERE text LIKE \"%KLM%\" 
"""

query_BA_sent = """
SELECT *
FROM main
WHERE "User ID" = "18332190"
"""

query_KLM_sent = """
SELECT *
FROM main
WHERE "User ID" = "56377143"
"""

df_ba_total = pd.read_sql_query(query_BA_total, conn)
df_klm_total = pd.read_sql_query(query_KLM_total, conn)
df_ba_sent = pd.read_sql_query(query_BA_sent, conn)
df_klm_sent = pd.read_sql_query(query_KLM_sent, conn)

In [ ]:
query_KLM_test = """
SELECT *
FROM main
WHERE text LIKE \"%KLM%\" AND text LIKE \"%fucking%\" AND text LIKE \"%love%\"
"""
df_klm_test = pd.read_sql_query(query_KLM_test, conn)

In [ ]:
df_klm_test

In [ ]:
for i in df_klm_test["Text"]:
    print(i)

In [ ]:
df_time_tweets_klm = df_klm_total[["Created At", "Text"]].copy()
df_time_tweets_ba = df_ba_total[["Created At", "Text"]].copy()

In [ ]:
df_BA_complete = df_BA_complete[["tweet_volume", "date"]].copy()
df_KLM_complete = df_KLM_complete[["tweet_volume", "date"]].copy()

In [ ]:
#heatmaps for tweet volume KLM compared to BA
fig, ax = plt.subplots(nrows=1, ncols=2, squeeze=False, figsize = (20, 5))

df_test_KLM = df_KLM_complete.copy()
df_test_KLM["date_test"] = pd.to_datetime(df_test_KLM["date"])
df_test_KLM["Month"] = df_test_KLM["date_test"].apply(lambda x:x.strftime("%b"))
df_test_KLM["Day"] = df_test_KLM["date_test"].apply(lambda x:x.day)

matrix_heatmap_KLM = df_test_KLM.pivot("Month", "Day", "tweet_volume")
levels_heatmap_KLM = [i[:3] for i in ["May", "June", "July", "August", "September", "October", "November", "December","January", "February", "March"]]
matrix_heatmap_KLM = matrix_heatmap_KLM.reindex_axis(axis = 0, labels = levels_heatmap_KLM)

sns.heatmap(matrix_heatmap_KLM, ax = ax[0,0], vmin = 0, vmax = 20000)
ax[0,0].set_title("Tweet volume for tweets related to KLM per month per day", size=16, weight='bold')
ax[0,0].set_ylabel("Month")
ax[0,0].set_xlabel("Day");

df_test = df_BA_complete.copy()
df_test["date_test"] = pd.to_datetime(df_test["date"])
df_test["Month"] = df_test["date_test"].apply(lambda x:x.strftime("%b"))
df_test["Day"] = df_test["date_test"].apply(lambda x:x.day)

matrix_heatmap = df_test.pivot("Month", "Day", "tweet_volume")
levels_heatmap = [i[:3] for i in ["May", "June", "July", "August", "September", "October", "November", "December","January", "February", "March"]]
matrix_heatmap = matrix_heatmap.reindex_axis(axis = 0, labels = levels_heatmap)

sns.heatmap(matrix_heatmap, ax = ax[0, 1],  vmax = 20000)
ax[0,1].set_title("Tweet volume for tweets related to British Airways per month per day", size=16, weight='bold')
ax[0,1].set_ylabel("Month")
ax[0,1].set_xlabel("Day");

In [ ]:
#Code for conversation

In [ ]:
#selecting all the tweets with involvement from KLM
conversations_KLM = """
SELECT "Reply User", replies.ID, replies."Reply ID", "User ID", Date("Created At") as Date
FROM replies, main
WHERE main.ID = replies.ID AND ("Reply User" = 56377143 or main."User ID" = 56377143) 
ORDER BY "Created At" DESC
"""
df_conversations_KLM = pd.read_sql_query(conversations_KLM, conn)

In [ ]:
#Splitting everything based on the month for KLM
may_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-05-01') & (df_conversations_KLM["Date"]<= "2016-05-31")]
june_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-06-01') & (df_conversations_KLM["Date"]<= "2016-06-31")]
july_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-07-01') & (df_conversations_KLM["Date"]<= "2016-07-31")]
aug_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-08-01') & (df_conversations_KLM["Date"]<= "2016-08-31")]
sep_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-09-01') & (df_conversations_KLM["Date"]<= "2016-09-31")]
oct_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-10-01') & (df_conversations_KLM["Date"]<= "2016-10-31")]
nov_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-11-01') & (df_conversations_KLM["Date"]<= "2016-11-31")]
dec_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2016-12-01') & (df_conversations_KLM["Date"]<= "2016-12-31")]
jan_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2017-01-01') & (df_conversations_KLM["Date"]<= "2017-01-31")]
feb_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2017-02-01') & (df_conversations_KLM["Date"]<= "2017-02-31")]
march_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2017-03-01') & (df_conversations_KLM["Date"]<= "2017-03-31")]
april_replies_df = df_conversations_KLM.loc[(df_conversations_KLM['Date'] >= '2017-04-01') & (df_conversations_KLM["Date"]<= "2017-04-31")]

may_replies_dict = may_replies_df.set_index('ID')['Reply ID'].to_dict()
june_replies_dict = june_replies_df.set_index('ID')['Reply ID'].to_dict()
july_replies_dict = july_replies_df.set_index('ID')['Reply ID'].to_dict()
aug_replies_dict = aug_replies_df.set_index('ID')['Reply ID'].to_dict()
sep_replies_dict = sep_replies_df.set_index('ID')['Reply ID'].to_dict()
oct_replies_dict = oct_replies_df.set_index('ID')['Reply ID'].to_dict()
nov_replies_dict = nov_replies_df.set_index('ID')['Reply ID'].to_dict()
dec_replies_dict = dec_replies_df.set_index('ID')['Reply ID'].to_dict()
jan_replies_dict = jan_replies_df.set_index('ID')['Reply ID'].to_dict()
feb_replies_dict = feb_replies_df.set_index('ID')['Reply ID'].to_dict()
march_replies_dict = march_replies_df.set_index('ID')['Reply ID'].to_dict()
april_replies_dict = april_replies_df.set_index('ID')['Reply ID'].to_dict()

In [ ]:
#All conversation enders
all_enders = """
SELECT replies.ID, "Reply ID", Date("Created At") as Date
FROM replies, main
WHERE replies.ID = main.ID AND replies.ID NOT IN (SELECT "Reply ID" from replies) 

"""
df_all_enders = pd.read_sql_query(all_enders, conn)

In [ ]:
#Splitting all the enders per month
may_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-05-01') & (df_all_enders["Date"]<= "2016-05-31")]
june_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-06-01') & (df_all_enders["Date"]<= "2016-06-31")]
july_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-07-01') & (df_all_enders["Date"]<= "2016-07-31")]
aug_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-08-01') & (df_all_enders["Date"]<= "2016-08-31")]
sep_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-09-01') & (df_all_enders["Date"]<= "2016-09-31")]
oct_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-10-01') & (df_all_enders["Date"]<= "2016-10-31")]
nov_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-11-01') & (df_all_enders["Date"]<= "2016-11-31")]
dec_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2016-12-01') & (df_all_enders["Date"]<= "2016-12-31")]
jan_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2017-01-01') & (df_all_enders["Date"]<= "2017-01-31")]
feb_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2017-02-01') & (df_all_enders["Date"]<= "2017-02-31")]
march_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2017-03-01') & (df_all_enders["Date"]<= "2017-03-31")]
april_ender_df = df_all_enders.loc[(df_all_enders['Date'] >= '2017-04-01') & (df_all_enders["Date"]<= "2017-04-31")]

ender_may = may_ender_df['ID'].tolist()
ender_june = june_ender_df['ID'].tolist()
ender_july = july_ender_df['ID'].tolist()
ender_aug = aug_ender_df['ID'].tolist()
ender_sep = sep_ender_df['ID'].tolist()
ender_oct = oct_ender_df['ID'].tolist()
ender_nov = nov_ender_df['ID'].tolist()
ender_dec = dec_ender_df['ID'].tolist()
ender_jan = jan_ender_df['ID'].tolist()
ender_feb = feb_ender_df['ID'].tolist()
ender_march = march_ender_df['ID'].tolist()
ender_april = april_ender_df['ID'].tolist()

In [ ]:
def month_con_dict (lst: list, replies: dict):
    """ takes ender_month as lst and month_replies_dict as replies
    returns a dictionary with all conversations of that month 
    """
    con_dict = {}
    for i in lst:
        if i in replies:
            con_dict[i] = []
    for k,v in con_dict.items():
        v.insert(0,k)
        current = k
        for i in replies:
            if current == i:
                current = replies[i]
                v.append(current)
    return con_dict
may_con_dict = month_con_dict (ender_may, may_replies_dict)
june_con_dict = month_con_dict (ender_june, june_replies_dict)
july_con_dict = month_con_dict (ender_july, july_replies_dict)
aug_con_dict = month_con_dict (ender_aug, aug_replies_dict)
sep_con_dict = month_con_dict (ender_sep, sep_replies_dict)
oct_con_dict = month_con_dict (ender_oct, oct_replies_dict)
nov_con_dict = month_con_dict (ender_nov, nov_replies_dict)
dec_con_dict = month_con_dict (ender_dec, dec_replies_dict)
jan_con_dict = month_con_dict (ender_jan, jan_replies_dict)
feb_con_dict = month_con_dict (ender_feb, feb_replies_dict)
march_con_dict = month_con_dict (ender_march, march_replies_dict)
april_con_dict = month_con_dict (ender_april, april_replies_dict)

In [ ]:
def last_first_klm (con_dict: dict):
    """takes month_con_dict
    returns a dictionary with last_first tweets in a conversation
    """
    con_user = {}
    for k ,v in con_dict.items():
        value_list = []
        con_user[k] = value_list
        for i in v:
            if i in tweet_user_dict:
                n = tweet_user_dict[i]
                value_list.append(n)
        if len(con_user[k]) < 3 or len(set(con_user[k])) != 2:
            del con_user[k]
    con_dict_final = {}
    for k, v in con_user.items():
        i = 0
        j = -1
        if v[i] == '56377143':
            i += 1
        if v[j] == '56377143':
            j -= 1
        n = '56377143'
        if n in v[i+1:j]:
            i = v.index(n)-1
            con_dict_final [k] = [con_dict[k][i],con_dict[k][j]]
    return con_dict_final
may_con_dict_final = last_first_klm(may_con_dict) 
june_con_dict_final = last_first_klm(june_con_dict)   
july_con_dict_final = last_first_klm(july_con_dict) 
aug_con_dict_final = last_first_klm(aug_con_dict) 
sep_con_dict_final = last_first_klm(sep_con_dict) 
oct_con_dict_final = last_first_klm(oct_con_dict) 
nov_con_dict_final = last_first_klm(nov_con_dict) 
dec_con_dict_final = last_first_klm(dec_con_dict) 
jan_con_dict_final = last_first_klm(jan_con_dict) 
feb_con_dict_final = last_first_klm(feb_con_dict) 
march_con_dict_final = last_first_klm(march_con_dict) 
april_con_dict_final = last_first_klm(april_con_dict) 

In [ ]:
def updated_lentgh (dictionary : dict):
    """average length and count of conversations
    input should be month_con_dict
    """
    con_length = 0
    count = 0
    for k,v in dictionary.items():
        if len(v) > 2:
            count += 1
            con_length +=  len(v)
    avr_len = con_length/ count
    return (avr_len, count)

In [ ]:
#Average conversation length and amount of conversations in May for KLM
try: 
    KLM_may_avg = updated_lentgh(may_con_dict)[0]
    KLM_may_total = updated_lentgh(may_con_dict)[1]
        
except: 
    KLM_may_avg = 0
    KLM_may_total = 0

In [ ]:
#Average conversation length and amount of conversations in June for KLM
try: 
    KLM_june_avg = updated_lentgh(june_con_dict)[0]
    KLM_june_total = updated_lentgh(june_con_dict)[1]
    
except:
    KLM_june_avg = 0
    KLM_june_total = 0

In [ ]:
#Average conversation length and amount of conversations in July for KLM
try:
    KLM_july_avg = updated_lentgh(july_con_dict)[0]
    KLM_july_total = updated_lentgh(july_con_dict)[1]
    
except:
    KLM_july_avg = 0
    KLM_july_total = 0

In [ ]:
#Average conversation length and amount of conversations in August for KLM
try:
    KLM_august_avg = updated_lentgh(aug_con_dict)[0]
    KLM_august_total = updated_lentgh(aug_con_dict)[1]

except:
    KLM_august_avg = 0
    KLM_august_total = 0

In [ ]:
#Average conversation length and amount of conversations in September for KLM
try:
    KLM_september_avg = updated_lentgh(sep_con_dict)[0]
    KLM_september_total = updated_lentgh(sep_con_dict)[1]

except:
    KLM_september_avg = 0
    KLM_september_total = 0

In [ ]:
#Average conversation length and amount of conversations in October for KLM
try:
    KLM_october_avg = updated_lentgh(oct_con_dict)[0]
    KLM_october_total = updated_lentgh(oct_con_dict)[1]

except:
    KLM_october_avg = 0
    KLM_october_total = 0

In [ ]:
#Average conversation length and amount of conversations in November for KLM
try: 
    KLM_november_avg = updated_lentgh(nov_con_dict)[0]
    KLM_november_total = updated_lentgh(nov_con_dict)[1]

except: 
    KLM_november_avg = 0
    KLM_november_total = 0

In [ ]:
#Average conversation length and amount of conversations in December for KLM
try: 
    KLM_december_avg = updated_lentgh(dec_con_dict)[0]
    KLM_december_total = updated_lentgh(dec_con_dict)[1]

except:
    KLM_december_avg = 0
    KLM_december_total = 0

In [ ]:
#Average conversation length and amount of conversations in January for KLM
try:
    KLM_january_avg = updated_lentgh(jan_con_dict)[0]
    KLM_january_total = updated_lentgh(jan_con_dict)[1]

except:
    KLM_january_avg = 0
    KLM_january_total = 0

In [ ]:
#Average conversation length and amount of conversations in February for KLM
try:
    KLM_february_avg = updated_lentgh(feb_con_dict)[0]
    KLM_february_total = updated_lentgh(feb_con_dict)[1]

except:
    KLM_february_avg = 0
    KLM_february_total = 0

In [ ]:
#Average conversation length and amount of conversations in March for KLM
try:
    KLM_march_avg = updated_lentgh(march_con_dict)[0]
    KLM_march_total = updated_lentgh(march_con_dict)[1]

except:
    KLM_march_avg = 0
    KLM_march_total = 0

In [ ]:
#Average conversation length and amount of conversations in April for KLM
try:
    KLM_april_avg = updated_lentgh(april_con_dict)[0]
    KLM_april_total = updated_lentgh(april_con_dict)[1]
    
except:
    KLM_april_avg = 0
    KLM_april_total = 0

In [ ]:
#Sentiment delta code

In [ ]:
sentiment_dict = df_complete.set_index('ID')['Sentiment'].to_dict()

In [ ]:
def sentiment_delta_init_end (dictionary):
    """ takes dictionary like june_con_dict_final
    returns a dictionary with conversations' enders with the sentiment delta they created
    """
    new_dict = {}
    for k in dictionary:
        if dictionary[k][0] in sentiment_dict and dictionary[k][1] in sentiment_dict:
            last_sent = sentiment_dict[dictionary[k][0]]
            first_sent = sentiment_dict[dictionary[k][1]]
            if type(last_sent) is float and type(first_sent) is float:
                new_dict[k] = [last_sent, first_sent]
    return (new_dict)

In [ ]:
def sentiment_new_test (lst):
    list_sentiment = []
    for i in lst:
        delta = i[1] - i[0]
        list_sentiment.append(delta)
        
    return list_sentiment

In [ ]:
#NEW CODE WITH CATEGORIES

In [ ]:
#May is empty
#in June 2 out of 13 go from positive to negative
#July 92 out of 1268
#August 86 out of 1130
#September 88 out of 1324
#October 55 out of 1113
#November 29 out of 672
#December 61 out of 982
#January 7 out of 140
#February 77 out of 944
#March 49 out of 746

In [ ]:
#June 15%
#July 7.1%
#August 7.6%
#September 6.6%
#October 4.9%
#November 4.3%
#December 6.2%
#January 5%
#February 8.2%
#March 6.6%

In [ ]:
may_positive_KLM = []
may_neutral_KLM = []
may_negative_KLM = []
may_sentiment_end_init = sentiment_delta_init_end(may_con_dict_final)
for i in may_sentiment_end_init:
    if may_sentiment_end_init[i][1] > 0:
        may_positive_KLM.append(may_sentiment_end_init[i])
    elif may_sentiment_end_init[i][1] < 0:
        may_negative_KLM.append(may_sentiment_end_init[i])
    else:
        may_neutral_KLM.append(may_sentiment_end_init[i])   
    
may_positive_senti_KLM = sentiment_new_test(may_positive_KLM)
may_negative_senti_KLM = sentiment_new_test(may_negative_KLM)
may_neutral_senti_KLM = sentiment_new_test(may_neutral_KLM)

In [ ]:
june_positive_KLM = []
june_neutral_KLM = []
june_negative_KLM = []
june_sentiment_end_init = sentiment_delta_init_end(june_con_dict_final)
for i in june_sentiment_end_init:
    if june_sentiment_end_init[i][1] > 0:
        june_positive_KLM.append(june_sentiment_end_init[i])
    elif june_sentiment_end_init[i][1] < 0:
        june_negative_KLM.append(june_sentiment_end_init[i])
    else:
        june_neutral_KLM.append(june_sentiment_end_init[i]) 
    
june_positive_senti_KLM = sentiment_new_test(june_positive_KLM)
june_negative_senti_KLM = sentiment_new_test(june_negative_KLM)
june_neutral_senti_KLM = sentiment_new_test(june_neutral_KLM)

In [ ]:
july_positive_KLM = []
july_neutral_KLM = []
july_negative_KLM = []
july_sentiment_end_init = sentiment_delta_init_end(july_con_dict_final)
for i in july_sentiment_end_init:
    if july_sentiment_end_init[i][1] > 0:
        july_positive_KLM.append(july_sentiment_end_init[i])
    elif july_sentiment_end_init[i][1] < 0:
        july_negative_KLM.append(july_sentiment_end_init[i])
    else:
        july_neutral_KLM.append(july_sentiment_end_init[i])  
        
july_positive_senti_KLM = sentiment_new_test(july_positive_KLM)
july_negative_senti_KLM = sentiment_new_test(july_negative_KLM)
july_neutral_senti_KLM = sentiment_new_test(july_neutral_KLM)

In [ ]:
august_positive_KLM = []
august_neutral_KLM = []
august_negative_KLM = []
august_sentiment_end_init = sentiment_delta_init_end(aug_con_dict_final)
for i in august_sentiment_end_init:
    if august_sentiment_end_init[i][1] > 0:
        august_positive_KLM.append(august_sentiment_end_init[i])
    elif august_sentiment_end_init[i][1] < 0:
        august_negative_KLM.append(august_sentiment_end_init[i])
    else:
        august_neutral_KLM.append(august_sentiment_end_init[i])  
        
august_positive_senti_KLM = sentiment_new_test(august_positive_KLM)
august_negative_senti_KLM = sentiment_new_test(august_negative_KLM)
august_neutral_senti_KLM = sentiment_new_test(august_neutral_KLM)

In [ ]:
september_positive_KLM = []
september_neutral_KLM = []
september_negative_KLM = []
september_sentiment_end_init = sentiment_delta_init_end(sep_con_dict_final)
for i in september_sentiment_end_init:
    if september_sentiment_end_init[i][1] > 0:
        september_positive_KLM.append(september_sentiment_end_init[i])
    elif september_sentiment_end_init[i][1] < 0:
        september_negative_KLM.append(september_sentiment_end_init[i])
    else:
        september_neutral_KLM.append(september_sentiment_end_init[i])  
        
september_positive_senti_KLM = sentiment_new_test(september_positive_KLM)
september_negative_senti_KLM = sentiment_new_test(september_negative_KLM)
september_neutral_senti_KLM = sentiment_new_test(september_neutral_KLM)

In [ ]:
october_positive_KLM = []
october_neutral_KLM = []
october_negative_KLM = []
october_sentiment_end_init = sentiment_delta_init_end(oct_con_dict_final)
for i in october_sentiment_end_init:
    if october_sentiment_end_init[i][1] > 0:
        october_positive_KLM.append(october_sentiment_end_init[i])
    elif october_sentiment_end_init[i][1] < 0:
        october_negative_KLM.append(october_sentiment_end_init[i])
    else:
        october_neutral_KLM.append(october_sentiment_end_init[i])  

october_positive_senti_KLM = sentiment_new_test(october_positive_KLM)
october_negative_senti_KLM = sentiment_new_test(october_negative_KLM)
october_neutral_senti_KLM = sentiment_new_test(october_neutral_KLM)

In [ ]:
november_positive_KLM = []
november_neutral_KLM = []
november_negative_KLM = []
november_sentiment_end_init = sentiment_delta_init_end(nov_con_dict_final)
for i in november_sentiment_end_init:
    if november_sentiment_end_init[i][1] > 0:
        november_positive_KLM.append(november_sentiment_end_init[i])
    elif november_sentiment_end_init[i][1] < 0:
        november_negative_KLM.append(november_sentiment_end_init[i])
    else:
        november_neutral_KLM.append(november_sentiment_end_init[i])  

november_positive_senti_KLM = sentiment_new_test(november_positive_KLM)
november_negative_senti_KLM = sentiment_new_test(november_negative_KLM)
november_neutral_senti_KLM = sentiment_new_test(november_neutral_KLM)

In [ ]:
december_positive_KLM = []
december_neutral_KLM = []
december_negative_KLM = []
december_sentiment_end_init = sentiment_delta_init_end(dec_con_dict_final)
for i in december_sentiment_end_init:
    if december_sentiment_end_init[i][1] > 0:
        december_positive_KLM.append(december_sentiment_end_init[i])
    elif december_sentiment_end_init[i][1] < 0:
        december_negative_KLM.append(december_sentiment_end_init[i])
    else:
        december_neutral_KLM.append(december_sentiment_end_init[i])  

december_positive_senti_KLM = sentiment_new_test(november_positive_KLM)
december_negative_senti_KLM = sentiment_new_test(november_negative_KLM)
december_neutral_senti_KLM = sentiment_new_test(november_neutral_KLM)

In [ ]:
january_positive_KLM = []
january_neutral_KLM = []
january_negative_KLM = []
january_sentiment_end_init = sentiment_delta_init_end(jan_con_dict_final)
for i in january_sentiment_end_init:
    if january_sentiment_end_init[i][1] > 0:
        january_positive_KLM.append(january_sentiment_end_init[i])
    elif january_sentiment_end_init[i][1] < 0:
        january_negative_KLM.append(january_sentiment_end_init[i])
    else:
        january_neutral_KLM.append(january_sentiment_end_init[i])  

january_positive_senti_KLM = sentiment_new_test(january_positive_KLM)
january_negative_senti_KLM = sentiment_new_test(january_negative_KLM)
january_neutral_senti_KLM = sentiment_new_test(january_neutral_KLM)

In [ ]:
february_positive_KLM = []
february_neutral_KLM = []
february_negative_KLM = []
february_sentiment_end_init = sentiment_delta_init_end(feb_con_dict_final)
for i in february_sentiment_end_init:
    if february_sentiment_end_init[i][1] > 0:
        february_positive_KLM.append(february_sentiment_end_init[i])
    elif february_sentiment_end_init[i][1] < 0:
        february_negative_KLM.append(february_sentiment_end_init[i])
    else:
        february_neutral_KLM.append(february_sentiment_end_init[i])  

february_positive_senti_KLM = sentiment_new_test(february_positive_KLM)
february_negative_senti_KLM = sentiment_new_test(february_negative_KLM)
february_neutral_senti_KLM = sentiment_new_test(february_neutral_KLM)

In [ ]:
march_positive_KLM = []
march_neutral_KLM = []
march_negative_KLM = []
march_sentiment_end_init = sentiment_delta_init_end(march_con_dict_final)
for i in march_sentiment_end_init:
    if march_sentiment_end_init[i][1] > 0:
        march_positive_KLM.append(march_sentiment_end_init[i])
    elif march_sentiment_end_init[i][1] < 0:
        march_negative_KLM.append(march_sentiment_end_init[i])
    else:
        march_neutral_KLM.append(march_sentiment_end_init[i])  

march_positive_senti_KLM = sentiment_new_test(march_positive_KLM)
march_negative_senti_KLM = sentiment_new_test(march_negative_KLM)
march_neutral_senti_KLM = sentiment_new_test(march_neutral_KLM)

In [ ]:
KLM_total_positive = may_positive_senti_KLM + june_positive_senti_KLM + july_positive_senti_KLM + august_positive_senti_KLM+september_positive_senti_KLM + october_positive_senti_KLM + november_positive_senti_KLM +december_positive_senti_KLM + january_positive_senti_KLM + february_positive_senti_KLM+march_positive_senti_KLM
KLM_total_negative = may_negative_senti_KLM + june_negative_senti_KLM + july_negative_senti_KLM + august_negative_senti_KLM+september_negative_senti_KLM + october_negative_senti_KLM + november_negative_senti_KLM +december_negative_senti_KLM + january_negative_senti_KLM + february_negative_senti_KLM+march_negative_senti_KLM
KLM_total_neutral = may_neutral_senti_KLM + june_neutral_senti_KLM + july_neutral_senti_KLM + august_neutral_senti_KLM+september_neutral_senti_KLM + october_neutral_senti_KLM + november_neutral_senti_KLM +december_neutral_senti_KLM + january_neutral_senti_KLM + february_neutral_senti_KLM+march_neutral_senti_KLM

In [ ]:
amount_positive_positive_KLM = 0
amount_positive_negative_KLM = 0
amount_positive_neutral_KLM = 0

for i in KLM_total_positive:
    if i < -0.25:
        amount_positive_negative_KLM += 1
    elif i > 0.25: 
        amount_positive_positive_KLM +=1
    else:
        amount_positive_neutral_KLM +=1

In [ ]:
amount_negative_positive_KLM = 0
amount_negative_negative_KLM = 0
amount_negative_neutral_KLM = 0

for i in KLM_total_negative:
    if i < -0.25:
        amount_negative_negative_KLM += 1
    elif i > 0.25: 
        amount_negative_positive_KLM +=1
    else:
        amount_negative_neutral_KLM +=1

In [ ]:
amount_neutral_positive_KLM = 0
amount_neutral_negative_KLM = 0
amount_neutral_neutral_KLM = 0

for i in KLM_total_neutral:
    if i < -0.25:
        amount_neutral_negative_KLM += 1
    elif i > 0.25: 
        amount_neutral_positive_KLM +=1
    else:
        amount_neutral_neutral_KLM +=1

In [ ]:
change_KLM = pd.DataFrame({'index' : ['positive', 'negative', 'neutral'],'positive': [amount_positive_positive_KLM, amount_positive_negative_KLM, amount_positive_neutral_KLM], 
                       'Negative': [amount_negative_positive_KLM, amount_negative_negative_KLM, amount_negative_neutral_KLM],
                     'Neutral' : [amount_neutral_positive_KLM, amount_neutral_negative_KLM, amount_neutral_neutral_KLM ]
                    })
change_KLM = change_KLM.set_index('index')

In [ ]:
#Code conversations BA

In [ ]:
#selecting all the tweets with involvement from BA
conversations_BA = """
SELECT "Reply User", replies.ID, replies."Reply ID", "User ID", Date("Created At") as Date
FROM replies, main
WHERE main.ID = replies.ID AND ("Reply User" = 18332190 or main."User ID" = 18332190) 
ORDER BY "Created At" DESC
"""
df_conversations_BA = pd.read_sql_query(conversations_BA, conn)

In [ ]:
#Splitting everything based on the month for BA
may_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-05-01') & (df_conversations_BA["Date"]<= "2016-05-31")]
june_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-06-01') & (df_conversations_BA["Date"]<= "2016-06-31")]
july_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-07-01') & (df_conversations_BA["Date"]<= "2016-07-31")]
aug_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-08-01') & (df_conversations_BA["Date"]<= "2016-08-31")]
sep_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-09-01') & (df_conversations_BA["Date"]<= "2016-09-31")]
oct_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-10-01') & (df_conversations_BA["Date"]<= "2016-10-31")]
nov_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-11-01') & (df_conversations_BA["Date"]<= "2016-11-31")]
dec_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2016-12-01') & (df_conversations_BA["Date"]<= "2016-12-31")]
jan_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2017-01-01') & (df_conversations_BA["Date"]<= "2017-01-31")]
feb_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2017-02-01') & (df_conversations_BA["Date"]<= "2017-02-31")]
march_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2017-03-01') & (df_conversations_BA["Date"]<= "2017-03-31")]
april_replies_df_BA = df_conversations_BA.loc[(df_conversations_BA['Date'] >= '2017-04-01') & (df_conversations_BA["Date"]<= "2017-04-31")]

may_replies_dict_BA = may_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
june_replies_dict_BA = june_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
july_replies_dict_BA = july_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
aug_replies_dict_BA = aug_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
sep_replies_dict_BA = sep_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
oct_replies_dict_BA = oct_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
nov_replies_dict_BA = nov_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
dec_replies_dict_BA = dec_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
jan_replies_dict_BA = jan_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
feb_replies_dict_BA = feb_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
march_replies_dict_BA = march_replies_df_BA.set_index('ID')['Reply ID'].to_dict()
april_replies_dict_BA = april_replies_df_BA.set_index('ID')['Reply ID'].to_dict()

In [ ]:
def month_con_dict_BA (lst: list, replies: dict):
    """ takes ender_month as lst and month_replies_dict_BA as replies
    returns a dictionary with all conversations of that month from BA
    """
    con_dict = {}
    for i in lst:
        if i in replies:
            con_dict[i] = []
    for k,v in con_dict.items():
        v.insert(0,k)
        current = k
        for i in replies:
            if current == i:
                current = replies[i]
                v.append(current)
    return con_dict
may_con_dict_BA = month_con_dict (ender_may, may_replies_dict_BA)
june_con_dict_BA = month_con_dict (ender_june, june_replies_dict_BA)
july_con_dict_BA = month_con_dict (ender_july, july_replies_dict_BA)
aug_con_dict_BA = month_con_dict (ender_aug, aug_replies_dict_BA)
sep_con_dict_BA = month_con_dict (ender_sep, sep_replies_dict_BA)
oct_con_dict_BA = month_con_dict (ender_oct, oct_replies_dict_BA)
nov_con_dict_BA = month_con_dict (ender_nov, nov_replies_dict_BA)
dec_con_dict_BA = month_con_dict (ender_dec, dec_replies_dict_BA)
jan_con_dict_BA = month_con_dict (ender_jan, jan_replies_dict_BA)
feb_con_dict_BA = month_con_dict (ender_feb, feb_replies_dict_BA)
march_con_dict_BA = month_con_dict (ender_march, march_replies_dict_BA)
april_con_dict_BA = month_con_dict (ender_april, april_replies_dict_BA)

In [ ]:
def last_first_BA (con_dict: dict):
    """takes month_con_dict_BA
    returns a dictionary with last_first tweets in a conversation
    """
    con_user = {}
    for k ,v in con_dict.items():
        value_list = []
        con_user[k] = value_list
        for i in v:
            if i in tweet_user_dict:
                n = tweet_user_dict[i]
                value_list.append(n)
        if len(con_user[k]) < 3 or len(set(con_user[k])) != 2:
            del con_user[k]
    con_dict_final = {}
    for k, v in con_user.items():
        i = 0
        j = -1
        if v[i] == '18332190':
            i += 1
        if v[j] == '18332190':
            j -= 1
        n = '18332190'
        if n in v[i+1:j]:
            i = v.index(n)-1
            con_dict_final [k] = [con_dict[k][i],con_dict[k][j]]
    return con_dict_final
may_con_dict_final_BA = last_first_BA(may_con_dict_BA) 
june_con_dict_final_BA = last_first_BA(june_con_dict_BA)   
july_con_dict_final_BA = last_first_BA(july_con_dict_BA) 
aug_con_dict_final_BA = last_first_BA(aug_con_dict_BA) 
sep_con_dict_final_BA = last_first_BA(sep_con_dict_BA) 
oct_con_dict_final_BA = last_first_BA(oct_con_dict_BA) 
nov_con_dict_final_BA = last_first_BA(nov_con_dict_BA) 
dec_con_dict_final_BA = last_first_BA(dec_con_dict_BA) 
jan_con_dict_final_BA = last_first_BA(jan_con_dict_BA) 
feb_con_dict_final_BA = last_first_BA(feb_con_dict_BA) 
march_con_dict_final_BA = last_first_BA(march_con_dict_BA) 
april_con_dict_final_BA = last_first_BA(april_con_dict_BA) 

In [ ]:
#Average conversation length and amount of conversations in May for BA
try: 
    BA_may_avg = updated_lentgh(may_con_dict_BA)[0]
    BA_may_total = updated_lentgh(may_con_dict_BA)[1]
        
except: 
    BA_may_avg = 0
    BA_may_total = 0

In [ ]:
#Average conversation length and amount of conversations in June for BA
try:
    BA_june_avg = updated_lentgh(june_con_dict_BA)[0]
    BA_june_total = updated_lentgh(june_con_dict_BA)[1]
    
except:
    BA_june_avg = 0
    BA_june_total = 0

In [ ]:
#Average conversation length and amount of conversations in July for BA
try:
    BA_july_avg = updated_lentgh(july_con_dict_BA)[0]
    BA_july_total = updated_lentgh(july_con_dict_BA)[1]

except:
    BA_july_avg = 0
    BA_july_total = 0 

In [ ]:
#Average conversation length and amount of conversations in August for BA
try:
    BA_august_avg = updated_lentgh(aug_con_dict_BA)[0]
    BA_august_total = updated_lentgh(aug_con_dict_BA)[1]
    
except:
    BA_august_avg = 0
    BA_august_total = 0

In [ ]:
#Average conversation length and amount of conversations in September for BA
try:
    BA_september_avg = updated_lentgh(sep_con_dict_BA)[0]
    BA_september_total = updated_lentgh(sep_con_dict_BA)[1]
    
except:
    BA_september_avg = 0
    BA_september_total = 0

In [ ]:
#Average conversation length and amount of conversations in October for BA
try:
    BA_october_avg = updated_lentgh(oct_con_dict_BA)[0]
    BA_october_total = updated_lentgh(oct_con_dict_BA)[1]
    
except:
    BA_october_avg = 0
    BA_october_total = 0

In [ ]:
#Average conversation length and amount of conversations in November for BA
try:
    BA_november_avg = updated_lentgh(nov_con_dict_BA)[0]
    BA_november_total = updated_lentgh(nov_con_dict_BA)[1]
    
except:
    BA_november_avg = 0
    BA_november_total = 0

In [ ]:
#Average conversation length and amount of conversations in December for BA
try:  
    BA_december_avg = updated_lentgh(dec_con_dict_BA)[0]
    BA_december_total = updated_lentgh(dec_con_dict_BA)[1]
    
except:
    BA_december_avg = 0
    BA_december_total = 0 

In [ ]:
#Average conversation length and amount of conversations in January for BA
try:
    BA_january_avg = updated_lentgh(jan_con_dict_BA)[0]
    BA_january_total = updated_lentgh(jan_con_dict_BA)[1]
    
except:
    BA_january_avg = 0
    BA_january_total = 0

In [ ]:
#Average conversation length and amount of conversations in February for BA
try:
    BA_february_avg = updated_lentgh(feb_con_dict_BA)[0]
    BA_february_total = updated_lentgh(feb_con_dict_BA)[1]
    
except:
    BA_february_avg = 0
    BA_february_total = 0

In [ ]:
#Average conversation length and amount of conversations in March for BA
try:
    BA_march_avg = updated_lentgh(march_con_dict_BA)[0]
    BA_march_total = updated_lentgh(march_con_dict_BA)[1]
    
except:
    BA_march_avg = 0
    BA_march_total = 0

In [ ]:
#Average conversation length and amount of conversations in April for BA
try:
    BA_april_avg = updated_lentgh(april_con_dict_BA)[0]
    BA_april_total = updated_lentgh(april_con_dict_BA)[1]
    
except:
    BA_april_avg = 0
    BA_april_total = 0

In [ ]:
#Sentiment delta code BA

In [ ]:
may_positive_BA = []
may_neutral_BA = []
may_negative_BA = []
may_sentiment_end_init_BA = sentiment_delta_init_end(may_con_dict_final_BA)
for i in may_sentiment_end_init_BA:
    if may_sentiment_end_init_BA[i][1] > 0:
        may_positive_BA.append(may_sentiment_end_init_BA[i])
    elif may_sentiment_end_init_BA[i][1] < 0:
        may_negative_BA.append(may_sentiment_end_init_BA[i])
    else:
        may_neutral_BA.append(may_sentiment_end_init_BA[i])   
    
may_positive_senti_BA = sentiment_new_test(may_positive_BA)
may_negative_senti_BA = sentiment_new_test(may_negative_BA)
may_neutral_senti_BA = sentiment_new_test(may_neutral_BA)

In [ ]:
june_positive_BA = []
june_neutral_BA = []
june_negative_BA = []
june_sentiment_end_init_BA = sentiment_delta_init_end(june_con_dict_final_BA)
for i in june_sentiment_end_init_BA:
    if june_sentiment_end_init_BA[i][1] > 0:
        june_positive_BA.append(june_sentiment_end_init_BA[i])
    elif june_sentiment_end_init_BA[i][1] < 0:
        june_negative_KLM_BA.append(june_sentiment_end_init_BA[i])
    else:
        june_neutral_BA.append(june_sentiment_end_init_BA[i]) 
    
june_positive_senti_BA = sentiment_new_test(june_positive_BA)
june_negative_senti_BA = sentiment_new_test(june_negative_BA)
june_neutral_senti_BA = sentiment_new_test(june_neutral_BA)

In [ ]:
july_positive_BA = []
july_neutral_BA = []
july_negative_BA = []
july_sentiment_end_init_BA = sentiment_delta_init_end(july_con_dict_final_BA)
for i in july_sentiment_end_init_BA:
    if july_sentiment_end_init_BA[i][1] > 0:
        july_positive_BA.append(july_sentiment_end_init_BA[i])
    elif july_sentiment_end_init_BA[i][1] < 0:
        july_negative_BA.append(july_sentiment_end_init_BA[i])
    else:
        july_neutral_BA.append(july_sentiment_end_init_BA[i])  
        
july_positive_senti_BA = sentiment_new_test(july_positive_BA)
july_negative_senti_BA = sentiment_new_test(july_negative_BA)
july_neutral_senti_BA = sentiment_new_test(july_neutral_BA)

In [ ]:
august_positive_BA = []
august_neutral_BA = []
august_negative_BA = []
august_sentiment_end_init_BA = sentiment_delta_init_end(aug_con_dict_final_BA)
for i in august_sentiment_end_init_BA:
    if august_sentiment_end_init_BA[i][1] > 0:
        august_positive_BA.append(august_sentiment_end_init_BA[i])
    elif august_sentiment_end_init_BA[i][1] < 0:
        august_negative_BA.append(august_sentiment_end_init_BA[i])
    else:
        august_neutral_BA.append(august_sentiment_end_init_BA[i])  
        
august_positive_senti_BA = sentiment_new_test(august_positive_BA)
august_negative_senti_BA = sentiment_new_test(august_negative_BA)
august_neutral_senti_BA = sentiment_new_test(august_neutral_BA)

In [ ]:
september_positive_BA = []
september_neutral_BA = []
september_negative_BA = []
september_sentiment_end_init_BA = sentiment_delta_init_end(sep_con_dict_final_BA)
for i in september_sentiment_end_init_BA:
    if september_sentiment_end_init_BA[i][1] > 0:
        september_positive_BA.append(september_sentiment_end_init_BA[i])
    elif september_sentiment_end_init_BA[i][1] < 0:
        september_negative_BA.append(september_sentiment_end_init_BA[i])
    else:
        september_neutral_BA.append(september_sentiment_end_init_BA[i])  
        
september_positive_senti_BA = sentiment_new_test(september_positive_BA)
september_negative_senti_BA = sentiment_new_test(september_negative_BA)
september_neutral_senti_BA = sentiment_new_test(september_neutral_BA)

In [ ]:
october_positive_BA = []
october_neutral_BA = []
october_negative_BA = []
october_sentiment_end_init_BA = sentiment_delta_init_end(oct_con_dict_final_BA)
for i in october_sentiment_end_init_BA:
    if october_sentiment_end_init_BA[i][1] > 0:
        october_positive_BA.append(october_sentiment_end_init_BA[i])
    elif october_sentiment_end_init_BA[i][1] < 0:
        october_negative_BA.append(october_sentiment_end_init_BA[i])
    else:
        october_neutral_BA.append(october_sentiment_end_init_BA[i])  

october_positive_senti_BA = sentiment_new_test(october_positive_BA)
october_negative_senti_BA = sentiment_new_test(october_negative_BA)
october_neutral_senti_BA = sentiment_new_test(october_neutral_BA)

In [ ]:
november_positive_BA = []
november_neutral_BA = []
november_negative_BA = []
november_sentiment_end_init_BA = sentiment_delta_init_end(nov_con_dict_final_BA)
for i in november_sentiment_end_init_BA:
    if november_sentiment_end_init_BA[i][1] > 0:
        november_positive_BA.append(november_sentiment_end_init_BA[i])
    elif november_sentiment_end_init_BA[i][1] < 0:
        november_negative_BA.append(november_sentiment_end_init_BA[i])
    else:
        november_neutral_BA.append(november_sentiment_end_init_BA[i])  

november_positive_senti_BA = sentiment_new_test(november_positive_BA)
november_negative_senti_BA = sentiment_new_test(november_negative_BA)
november_neutral_senti_BA = sentiment_new_test(november_neutral_BA)

In [ ]:
december_positive_BA = []
december_neutral_BA = []
december_negative_BA = []
december_sentiment_end_init_BA = sentiment_delta_init_end(dec_con_dict_final_BA)
for i in december_sentiment_end_init_BA:
    if december_sentiment_end_init_BA[i][1] > 0:
        december_positive_BA.append(december_sentiment_end_init_BA[i])
    elif december_sentiment_end_init_BA[i][1] < 0:
        december_negative_BA.append(december_sentiment_end_init_BA[i])
    else:
        december_neutral_BA.append(december_sentiment_end_init_BA[i])  

december_positive_senti_BA = sentiment_new_test(november_positive_BA)
december_negative_senti_BA = sentiment_new_test(november_negative_BA)
december_neutral_senti_BA = sentiment_new_test(november_neutral_BA)

In [ ]:
january_positive_BA = []
january_neutral_BA = []
january_negative_BA = []
january_sentiment_end_init_BA = sentiment_delta_init_end(jan_con_dict_final_BA)
for i in january_sentiment_end_init_BA:
    if january_sentiment_end_init_BA[i][1] > 0:
        january_positive_BA.append(january_sentiment_end_init_BA[i])
    elif january_sentiment_end_init_BA[i][1] < 0:
        january_negative_BA.append(january_sentiment_end_init_BA[i])
    else:
        january_neutral_BA.append(january_sentiment_end_init_BA[i])  

january_positive_senti_BA = sentiment_new_test(january_positive_BA)
january_negative_senti_BA = sentiment_new_test(january_negative_BA)
january_neutral_senti_BA = sentiment_new_test(january_neutral_BA)

In [ ]:
february_positive_BA = []
february_neutral_BA = []
february_negative_BA = []
february_sentiment_end_init_BA = sentiment_delta_init_end(feb_con_dict_final_BA)
for i in february_sentiment_end_init_BA:
    if february_sentiment_end_init_BA[i][1] > 0:
        february_positive_BA.append(february_sentiment_end_init_BA[i])
    elif february_sentiment_end_init_BA[i][1] < 0:
        february_negative_BA.append(february_sentiment_end_init_BA[i])
    else:
        february_neutral_BA.append(february_sentiment_end_init_BA[i])  

february_positive_senti_BA = sentiment_new_test(february_positive_BA)
february_negative_senti_BA = sentiment_new_test(february_negative_BA)
february_neutral_senti_BA = sentiment_new_test(february_neutral_BA)

In [ ]:
march_positive_BA = []
march_neutral_BA = []
march_negative_BA = []
march_sentiment_end_init_BA = sentiment_delta_init_end(march_con_dict_final_BA)
for i in march_sentiment_end_init_BA:
    if march_sentiment_end_init_BA[i][1] > 0:
        march_positive_BA.append(march_sentiment_end_init_BA[i])
    elif march_sentiment_end_init_BA[i][1] < 0:
        march_negative_BA.append(march_sentiment_end_init_BA[i])
    else:
        march_neutral_BA.append(march_sentiment_end_init_BA[i])  

march_positive_senti_BA = sentiment_new_test(march_positive_BA)
march_negative_senti_BA = sentiment_new_test(march_negative_BA)
march_neutral_senti_BA = sentiment_new_test(march_neutral_BA)

In [ ]:
BA_total_positive = may_positive_senti_BA + june_positive_senti_BA + july_positive_senti_BA + august_positive_senti_BA+september_positive_senti_BA + october_positive_senti_BA + november_positive_senti_BA +december_positive_senti_BA + january_positive_senti_BA + february_positive_senti_BA+march_positive_senti_BA
BA_total_negative = may_negative_senti_BA + june_negative_senti_BA + july_negative_senti_BA + august_negative_senti_BA+september_negative_senti_BA+ october_negative_senti_BA + november_negative_senti_BA +december_negative_senti_BA + january_negative_senti_BA + february_negative_senti_BA+march_negative_senti_BA
BA_total_neutral = may_neutral_senti_BA + june_neutral_senti_BA + july_neutral_senti_BA + august_neutral_senti_BA+september_neutral_senti_BA + october_neutral_senti_BA + november_neutral_senti_BA +december_neutral_senti_BA + january_neutral_senti_BA + february_neutral_senti_BA+march_neutral_senti_BA

In [ ]:
amount_positive_positive_BA = 0
amount_positive_negative_BA = 0
amount_positive_neutral_BA = 0

for i in BA_total_positive:
    if i < -0.25:
        amount_positive_negative_BA += 1
    elif i > 0.25: 
        amount_positive_positive_BA +=1
    else:
        amount_positive_neutral_BA +=1

In [ ]:
amount_negative_positive_BA = 0
amount_negative_negative_BA = 0
amount_negative_neutral_BA = 0

for i in BA_total_negative:
    if i < -0.25:
        amount_negative_negative_BA += 1
    elif i > 0.25: 
        amount_negative_positive_BA +=1
    else:
        amount_negative_neutral_BA +=1

In [ ]:
amount_neutral_positive_BA = 0
amount_neutral_negative_BA = 0
amount_neutral_neutral_BA = 0

for i in BA_total_neutral:
    if i < -0.25:
        amount_neutral_negative_BA += 1
    elif i > 0.25: 
        amount_neutral_positive_BA +=1
    else:
        amount_neutral_neutral_BA +=1

In [ ]:
change_BA = pd.DataFrame({'index' : ['positive', 'negative', 'neutral'],'positive': [amount_positive_positive_BA, amount_positive_negative_BA, amount_positive_neutral_BA], 
                       'Negative': [amount_negative_positive_BA, amount_negative_negative_BA, amount_negative_neutral_BA],
                     'Neutral' : [amount_neutral_positive_BA, amount_neutral_negative_BA, amount_neutral_neutral_BA ]
                    })
change_BA = change_BA.set_index('index')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, squeeze=False, figsize = (15, 5))

sns.heatmap(change_KLM, ax = ax[0,0], square = True, vmax = 10000)
ax[0,0].set_title("Sentiment change for KLM", size=14, weight='bold')
ax[0,0].set_ylabel('Sentiment delta')
ax[0,0].set_xlabel('Initial Sentiment');

sns.heatmap(change_BA, ax = ax[0,1], square = True, vmax = 10000)
ax[0,1].set_title("Sentiment change for British Airways", size=14, weight='bold')
ax[0,1].set_ylabel('Sentiment delta')
ax[0,1].set_xlabel('Initial Sentiment');

In [ ]:
KLM_list_avg = [KLM_may_avg, KLM_june_avg, KLM_july_avg, KLM_august_avg, KLM_september_avg, KLM_october_avg, KLM_november_avg, 
               KLM_december_avg, KLM_january_avg, KLM_february_avg, KLM_march_avg]

KLM_list_total = [KLM_may_total, KLM_june_total, KLM_july_total, KLM_august_total, KLM_september_total, KLM_october_total, 
                 KLM_november_total, KLM_december_total, KLM_january_total, KLM_february_total, KLM_march_total]

BA_list_avg = [BA_may_avg, BA_june_avg, BA_july_avg, BA_august_avg, BA_september_avg, BA_october_avg, BA_november_avg, 
               BA_december_avg, BA_january_avg, BA_february_avg, BA_march_avg]

BA_list_total = [BA_may_total, BA_june_total, BA_july_total, BA_august_total, BA_september_total, BA_october_total, 
                 BA_november_total, BA_december_total, BA_january_total, BA_february_total, BA_march_total]

months = ["May", "June", "July", "August", "September", "October", "November", "December","January", "February", "March"]

In [ ]:
# index = np.arange(11)
# bar_width = 0.35
# fig, ax = plt.subplots()
# bar_KLM = plt.bar(index, KLM_list_avg, bar_width, color = 'blue', label = 'KLM')
# bar_BA = plt.bar(index + bar_width, BA_list_avg, bar_width, color = 'orange', label = 'BA' )
# plt.xticks(index + (0.5 * bar_width), (months))
# plt.legend()

# ax.set_title("Average conversation length for KLM and BA per month", size=16, weight='bold')
# ax.set_ylabel("Average length")

# ax.set_xlabel("Month");

In [ ]:
# index = np.arange(11)
# bar_width = 0.35
# fig, ax = plt.subplots()
# bar_KLM = plt.bar(index, KLM_list_total, bar_width, color = 'blue', label = 'KLM')
# bar_BA = plt.bar(index + bar_width, BA_list_total, bar_width, color = 'orange', label = 'BA' )
# plt.xticks(index + (0.5 * bar_width), (months))
# plt.legend()

# ax.set_title("Amount of conversations for KLM and BA per month", size=16, weight='bold')
# ax.set_ylabel("Total amount")

# ax.set_xlabel("Month");

In [ ]:
#Sprint 2

In [ ]:
#converting all the dates to weekdays
def to_weekday(t):
    if (t.dayofweek == 0):
        return "Monday"
    elif (t.dayofweek == 1):
        return "Tuesday"
    elif (t.dayofweek == 2):
        return "Wednesday"
    elif (t.dayofweek == 3):
        return "Thursday"
    elif (t.dayofweek == 4):
        return "Friday"
    elif (t.dayofweek == 5):
        return "Saturday"
    elif (t.dayofweek == 6):
        return "Sunday"

In [ ]:
#Extra function to make the heatmap looks more organized
def to_weekday_heatmap(t):
    if (t.dayofweek == 0):
        return "0 Monday"
    elif (t.dayofweek == 1):
        return "1 Tuesday"
    elif (t.dayofweek == 2):
        return "2 Wednesday"
    elif (t.dayofweek == 3):
        return "3 Thursday"
    elif (t.dayofweek == 4):
        return "4 Friday"
    elif (t.dayofweek == 5):
        return "5 Saturday"
    elif (t.dayofweek == 6):
        return "6 Sunday"

In [ ]:
#wintertime london
def time_category(t):
    if (t.hour >= 0) and (t.hour < 1):
        return 0
    elif (t.hour >= 1) and (t.hour < 2):
        return 1
    elif (t.hour >= 2) and (t.hour < 3):
        return 2
    elif (t.hour >= 3) and (t.hour < 4):
        return 3
    elif (t.hour >= 4) and (t.hour < 5):
        return 4
    elif (t.hour >= 5) and (t.hour < 6):
        return 5
    elif (t.hour >= 6) and (t.hour < 7):
        return 6
    elif (t.hour >= 7) and (t.hour < 8):
        return 7
    elif (t.hour >= 8) and (t.hour < 9):
        return 8
    elif (t.hour >= 9) and (t.hour < 10):
        return 9
    elif (t.hour >= 10) and (t.hour < 11):
        return 10
    elif (t.hour >= 11) and (t.hour < 12):
        return 11
    
    elif (t.hour >= 12) and (t.hour < 13):
        return 12
    elif (t.hour >= 13) and (t.hour < 14):
        return 13
    elif (t.hour >= 14) and (t.hour < 15):
        return 14
    elif (t.hour >= 15) and (t.hour < 16):
        return 15
    elif (t.hour >= 16) and (t.hour < 17):
        return 16
    elif (t.hour >= 17) and (t.hour < 18):
        return 17
    elif (t.hour >= 18) and (t.hour < 19):
        return 18
    elif (t.hour >= 19) and (t.hour < 20):
        return 19
    elif (t.hour >= 20) and (t.hour < 21):
        return 20
    elif (t.hour >= 21) and (t.hour < 22):
        return 21
    elif (t.hour >= 22) and (t.hour < 23):
        return 22
    else:
        return 23

In [ ]:
def time_category_KLM(t):
    if (t.hour >= 0) and (t.hour < 1):
        return 1
    elif (t.hour >= 1) and (t.hour < 2):
        return 2
    elif (t.hour >= 2) and (t.hour < 3):
        return 3
    elif (t.hour >= 3) and (t.hour < 4):
        return 4
    elif (t.hour >= 4) and (t.hour < 5):
        return 5
    elif (t.hour >= 5) and (t.hour < 6):
        return 6
    elif (t.hour >= 6) and (t.hour < 7):
        return 7
    elif (t.hour >= 7) and (t.hour < 8):
        return 8
    elif (t.hour >= 8) and (t.hour < 9):
        return 9
    elif (t.hour >= 9) and (t.hour < 10):
        return 10
    elif (t.hour >= 10) and (t.hour < 11):
        return 11
    elif (t.hour >= 11) and (t.hour < 12):
        return 12
    
    elif (t.hour >= 12) and (t.hour < 13):
        return 13
    elif (t.hour >= 13) and (t.hour < 14):
        return 14
    elif (t.hour >= 14) and (t.hour < 15):
        return 15
    elif (t.hour >= 15) and (t.hour < 16):
        return 16
    elif (t.hour >= 16) and (t.hour < 17):
        return 17
    elif (t.hour >= 17) and (t.hour < 18):
        return 18
    elif (t.hour >= 18) and (t.hour < 19):
        return 19
    elif (t.hour >= 19) and (t.hour < 20):
        return 20
    elif (t.hour >= 20) and (t.hour < 21):
        return 21
    elif (t.hour >= 21) and (t.hour < 22):
        return 22
    elif (t.hour >= 22) and (t.hour < 23):
        return 23
    else:
        return 0

In [ ]:
#Code and graphs for the percentages

In [ ]:
query_all = """
SELECT main.ID,Text,"Created At", "Reply ID", Date("Created At") as Date, "User ID"
FROM main, replies 
WHERE main.ID=replies.ID and ("User ID"= 56377143 or "User ID" = 18332190)
ORDER BY Datetime("Created At") desc
"""
df_response = pd.read_sql_query(query_all, conn)
df_response["Reply_timestamp"] = 0

In [ ]:
counter = 0
for key, value in df_response.iteritems():
    if key == "Reply ID":
        lst = value
        for i in lst:
            if str(i) in time_dict: 
                df_response.loc[counter, "Reply_timestamp"] = time_dict.get(i)
                counter += 1
            else:
                df_response.loc[counter, "Reply_timestamp"] = 0
                counter += 1

In [ ]:
df_replies = df_response[df_response["Reply_timestamp"] != 0]

In [ ]:
list_time_delta = []
for row in df_replies[["Created At", "Reply_timestamp"]].iterrows():
    time = row[1]
    datetime_object1 = datetime.datetime.strptime(time[0], '%Y-%m-%d %H:%M:%S+00:00')
    datetime_object2 = datetime.datetime.strptime(time[1], '%Y-%m-%d %H:%M:%S+00:00')
    
    delta = abs(datetime_object1- datetime_object2)
    list_time_delta += [delta]

In [ ]:
df_replies["Time delta"]= list_time_delta

In [ ]:
df_replies["Date"] = pd.to_datetime(df_replies["Created At"])

df_replies_klm = df_replies[df_replies["User ID"] == "56377143"] 
df_replies_ba = df_replies[df_replies["User ID"] == "18332190"]  

df_replies_klm["Time_category"] = [time_category_KLM(x) for x in df_replies_klm["Date"]]
df_replies_ba["Time_category"] = [time_category(x) for x in df_replies_ba["Date"]]

In [ ]:
df_replies_klm["Minutes"] = [x.hour *60 + x.minute +x.second/60 for x in pd.to_datetime(df_replies_klm["Time delta"])]
df_replies_ba["Minutes"] = [x.hour *60 + x.minute +x.second/60 for x in pd.to_datetime(df_replies_ba["Time delta"])]

In [ ]:
df_plot_klm = pd.DataFrame(df_replies_klm.groupby(['Time_category'])["Minutes"].mean())
df_plot_ba = pd.DataFrame(df_replies_ba.groupby(['Time_category'])["Minutes"].mean())

In [ ]:
df_time_tweets_klm["Date"] = pd.to_datetime(df_time_tweets_klm["Created At"])
df_time_tweets_klm["Time_category"] = [time_category_KLM(x) for x in df_time_tweets_klm["Date"]]

In [ ]:
df_time_tweets_ba["Date"] = pd.to_datetime(df_time_tweets_ba["Created At"])
df_time_tweets_ba["Time_category"] = [time_category(x) for x in df_time_tweets_ba["Date"]]

In [ ]:
df_time_category_klm = pd.DataFrame(df_time_tweets_klm.groupby(['Time_category']).count())
df_time_category_ba = pd.DataFrame(df_time_tweets_ba.groupby(['Time_category']).count())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = [list(df_plot_klm.index)] #hours
s = [n for n in df_plot_klm["Minutes"]]
y = list(df_time_category_klm["Created At"])

x2 = [list(df_plot_ba.index)] #hours
s2 = [n for n in df_plot_ba["Minutes"]]
y2 = list(df_time_category_ba["Created At"])

ax.scatter(x2, y2, s2, color ="Orange")
ax.scatter(x, y, s, color ="#0000FF")

ba_legend = mpatches.Patch(color='Orange', label='BA')
klm_legend = mpatches.Patch(color='#0000FF', label='KLM')

for time in [50, 150, 300, 450]:
    plt.scatter([], [], c='orange', alpha=0.3, s=time,
                label=str(time) + ' Mins')

leg1 = ax.legend(loc='upper left', scatterpoints=1, frameon=False, labelspacing=1, title='Response time')
leg2 = ax.legend(handles = [ba_legend, klm_legend], title = "Airline", loc = 'upper right')

ax.add_artist(leg1)
txt="Hours of the day \n \n This plot shows the total number of tweets per hour of the day on the x and y axes. \n The colors indicate the airline and the size of the points show the response time"
plt.xlabel(txt)
plt.ylabel("Number of tweets")
plt.title("Total number of tweets and response time per hour of the day per airline", size = 16, weight = 'bold');

In [ ]:
df_ba_total["Date"] = pd.to_datetime(df_ba_total["Created At"])
df_ba_total["Time_category"] = [time_category(x) for x in df_ba_total["Date"]]

df_klm_total["Date"] = pd.to_datetime(df_klm_total["Created At"])
df_klm_total["Time_category"] = [time_category_KLM(x) for x in df_klm_total["Date"]]

df_ba_sent["Date"] = pd.to_datetime(df_ba_total["Created At"])
df_ba_sent["Time_category"] = [time_category(x) for x in df_ba_sent["Date"]]

df_klm_sent["Date"] = pd.to_datetime(df_klm_total["Created At"])
df_klm_sent["Time_category"] = [time_category(x) for x in df_klm_sent["Date"]]

klm_sent = df_klm_sent.groupby(['Time_category']).count()

ba_sent = df_ba_sent.groupby(['Time_category']).count()

klm_all = df_klm_total.groupby(['Time_category']).count()

ba_all = df_ba_total.groupby(['Time_category']).count()

percentage_ba = pd.DataFrame(ba_sent["ID"]/ba_all["ID"])
percentage_klm = pd.DataFrame(klm_sent["ID"]/klm_all["ID"])

In [ ]:
index = np.arange(percentage_ba.shape[0])
hours = [n for n in range(percentage_ba.shape[0])]
bar_width = 0.35
fig, ax = plt.subplots()
bar_KLM_perc = plt.bar(index, list(percentage_klm["ID"]), bar_width, color = 'blue', label = 'KLM')
bar_BA_perc = plt.bar(index + bar_width, list(percentage_ba["ID"]), bar_width, color = 'orange', label = 'BA' )

plt.xticks(index + (0.5 * bar_width), hours)
plt.legend()

ax.set_title("Tweets by KLM and BA: percentage of their tweet volume", size=16, weight='bold')
ax.set_ylabel("Percentage")

ax.set_xlabel("Hour of the day");

In [ ]:
sent_total_comparison = pd.DataFrame({'Positive': [len(KLM_total_positive), len(BA_total_positive)], 
                              'Negative': [len(KLM_total_negative),  len(BA_total_negative)], 
                              'Neutral': [len(KLM_total_neutral),  len(BA_total_neutral)]}, 
                                    index = pd.Index(['KLM', 'BA'],name = 'Airline' ))

sent_total_comparison

In [ ]:
barWidth = 0.25

total_KLM = len(KLM_total_positive) + len(KLM_total_negative) + len(KLM_total_neutral)
total_BA = len(BA_total_positive) + len(BA_total_negative) + len(BA_total_neutral)

bar1 = [len(KLM_total_positive)/total_KLM, len(KLM_total_negative)/total_KLM, len(KLM_total_neutral)/total_KLM]
bar2 = [len(BA_total_positive)/total_BA, len(BA_total_negative)/total_BA, len(BA_total_neutral)/total_BA]

r1 = np.arange(len(bar1))
r2 = [x + barWidth for x in r1]

plt.bar(r1, bar1, color='blue', width=barWidth, edgecolor='white', label='KLM')
plt.bar(r2, bar2, color='orange', width=barWidth, edgecolor='white', label='BA')

plt.xlabel('Sentiment category')
plt.ylabel("Percentage")
plt.title("Percentage tweets per category for KLM and BA", fontsize = 16, fontweight = "bold")
plt.xticks([r + barWidth for r in range(len(bar1))], ['Positive', 'Negative', 'Neutral'])

plt.legend()
plt.show()